<a href="https://colab.research.google.com/github/derricksobrien/101-tutorial/blob/master/Subprocess_Diagnostics_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import sys
import platform

# --- Configuration for Cross-Platform Compatibility ---
# Define common commands based on the operating system to ensure the script runs
# successfully on both Windows and Linux/macOS.
if platform.system() == "Windows":
    # Windows commands
    CMD_SUCCESS = ["cmd", "/c", "echo Success && exit 0"]
    CMD_FAIL = ["cmd", "/c", "nonexistentcommand123"]
    CMD_STDOUT = ["ipconfig"]  # Outputs network configuration
    CMD_STDIN = ["sort"]  # Windows command to sort lines from STDIN
    CMD_GREP_FAIL = ["findstr", "NONEXISTENT_TEXT"] # Command that exits non-zero if text not found
    CMD_ECHO = ["echo"]
else:
    # Linux/macOS commands
    CMD_SUCCESS = ["sh", "-c", "exit 0"]
    CMD_FAIL = ["nonexistentcommand123"]
    CMD_STDOUT = ["ls", "-l"]
    CMD_STDIN = ["cat"] # Command to read from STDIN and output
    CMD_GREP_FAIL = ["grep", "NONEXISTENT_TEXT"] # Command that exits non-zero if text not found
    CMD_ECHO = ["echo"]


print(f"--- Running Subprocess Exercise on {platform.system()} ---")

# ====================================================================
# Part 1: Running Commands and Checking Status (Exit Codes)
# ====================================================================
print("\n[PART 1: EXIT CODES]")

# 1. Command 1 (Success)
try:
    print(f"\n1. Executing successful command: {' '.join(CMD_SUCCESS)}")
    result_success = subprocess.run(CMD_SUCCESS, capture_output=True, text=True)

    print(f"   Exit Code: {result_success.returncode}")
    if result_success.returncode == 0:
        print("   Result: Command SUCCEEDED.")
    else:
        print("   Result: Command FAILED (Unexpected).")

except FileNotFoundError:
    print("   ERROR: Could not find the shell interpreter (sh or cmd). Skipping Part 1.")
    pass

# 2. Command 2 (Failure)
try:
    print(f"\n2. Executing failing command: {' '.join(CMD_FAIL)}")
    # We use 'shell=True' here only for demonstration in environments where
    # the command might not be found directly, to get the expected non-zero exit code
    # from the shell itself, which usually returns 127/1.
    result_failure = subprocess.run(CMD_FAIL, capture_output=True, text=True, check=False)

    print(f"   Exit Code: {result_failure.returncode}")
    if result_failure.returncode != 0:
        print("   Result: Command FAILED as expected (Non-zero exit code).")
    else:
        print("   Result: Command SUCCEEDED (Unexpected).")

except FileNotFoundError:
    # This block might catch errors if 'sh' or 'cmd' isn't found, but usually,
    # the command not found error is handled by the shell itself, resulting in an exit code.
    print("   ERROR: The command itself was not found in the system PATH.")
    pass


# ====================================================================
# Part 2: Managing STDOUT, STDERR, and Capturing Output
# ====================================================================
print("\n" + "="*50)
print("[PART 2: CAPTURING STDOUT AND STDERR]")

# 1. Execute a command that generates STDOUT and capture it
print(f"\n1. Capturing STDOUT from command: {' '.join(CMD_STDOUT)}")
try:
    # capture_output=True is a shortcut for setting stdout=PIPE and stderr=PIPE
    result_capture = subprocess.run(
        CMD_STDOUT,
        capture_output=True,
        text=True, # Decode stdout/stderr to strings automatically
        check=True
    )

    print(f"   Captured STDOUT (first 5 lines):\n{'-'*30}\n{'\n'.join(result_capture.stdout.splitlines()[:5])}\n{'-'*30}")
    print(f"   Captured STDERR: {'(Empty)' if not result_capture.stderr else result_capture.stderr}")

except subprocess.CalledProcessError as e:
    print(f"   ERROR: Command failed with exit code {e.returncode}. STDOUT: {e.stdout}, STDERR: {e.stderr}")
except FileNotFoundError:
    print("   ERROR: The command was not found in the system PATH. Skipping Part 2.")


# 2. Redirecting STDERR to STDOUT
# We use a command that is expected to fail or generate error output to demonstrate the redirection
print(f"\n2. Redirecting STDERR to STDOUT (using failing command for demo): {' '.join(CMD_GREP_FAIL)}")
try:
    result_redirect = subprocess.run(
        CMD_GREP_FAIL,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT, # CRITICAL: Redirects stderr to the same pipe as stdout
        text=True,
        check=False # Do not raise exception for non-zero exit code
    )

    print(f"   Captured Output (Contains both STDOUT and STDERR):\n{'-'*30}\n{result_redirect.stdout.strip()}\n{'-'*30}")
    print(f"   Check on STDERR object: {'(Empty as expected due to redirection)' if not result_redirect.stderr else result_redirect.stderr}")

except Exception as e:
    print(f"   An error occurred during STDERR redirection test: {e}")


# ====================================================================
# Part 3: Piping and STDIN
# ====================================================================
print("\n" + "="*50)
print("[PART 3: MANAGING STDIN (Simulating Pipe)]")

input_data = "Apple\nBanana\nOrange\n"
# Note: For many system commands, the input needs to be bytes, so we encode the string.
# However, since we used text=True in Part 2, we should maintain consistency.
# In this case, `subprocess.run` handles the encoding/decoding automatically
# when `text=True` is used along with the `input` argument.

print(f"\n1. Executing command {' '.join(CMD_STDIN)} with STDIN input.")
print(f"   Input provided to STDIN: \n{input_data.strip().replace('\n', ', ')}")

try:
    result_stdin = subprocess.run(
        CMD_STDIN,
        input=input_data, # CRITICAL: This sends the string to the subprocess's STDIN
        capture_output=True,
        text=True,
        check=True
    )

    print(f"   Command Exit Code: {result_stdin.returncode}")
    print(f"   Captured STDOUT (Processed Input):\n{'-'*30}\n{result_stdin.stdout.strip()}\n{'-'*30}")
    print("   Verification: The output should match the input as the command just echoes it.")

except subprocess.CalledProcessError as e:
    print(f"   ERROR: Command failed with exit code {e.returncode}. STDOUT: {e.stdout}, STDERR: {e.stderr}")
except FileNotFoundError:
    print("   ERROR: The command was not found in the system PATH. Skipping Part 3.")

print("\n--- Exercise Complete ---")

--- Running Subprocess Exercise on Linux ---

[PART 1: EXIT CODES]

1. Executing successful command: sh -c exit 0
   Exit Code: 0
   Result: Command SUCCEEDED.

2. Executing failing command: nonexistentcommand123
   ERROR: The command itself was not found in the system PATH.

[PART 2: CAPTURING STDOUT AND STDERR]

1. Capturing STDOUT from command: ls -l
   Captured STDOUT (first 5 lines):
------------------------------
total 4
drwxr-xr-x 1 root root 4096 Nov  4 14:36 sample_data
------------------------------
   Captured STDERR: (Empty)

2. Redirecting STDERR to STDOUT (using failing command for demo): grep NONEXISTENT_TEXT
   Captured Output (Contains both STDOUT and STDERR):
------------------------------

------------------------------
   Check on STDERR object: (Empty as expected due to redirection)

[PART 3: MANAGING STDIN (Simulating Pipe)]

1. Executing command cat with STDIN input.
   Input provided to STDIN: 
Apple, Banana, Orange
   Command Exit Code: 0
   Captured STDOUT (Pr